# Modelo deposito de agua
En este Notebook vamos a crear un modelo muy simple en el que vamos a simular el llenado y vaciado de un deposito de agua que recibe agua de lluvia y que desaloja parte del agua acumulada por una tuberia de desagüe en el fondo del deposito.

## Caracteristicas del deposito
El deposito tiene una capacidad (`s_max`) de 10 litros y la tuberia de desagüe tiene una capacidad (`q_max`) de desalojar 3 litros al dia.

![](util/modelo_deposito_1.png)

In [ ]:
s_max = 10
q_max = 3

Imagina que tenemos una prediccion de la lluvia de los proximos 10 dias y nos gustaria saber que volumen de agua tendremos en el deposito cada dia. La prediccion de la lluvia consiste en un array `rain` con los 10 valores de lluvia para cada dia.

In [ ]:
import numpy as np # importamos la libreria NumPy
rain = np.array([3,15,2,2,6,0,1,0,0,5]) # Array con la prediccion de lluvia (litros / dia)

Nuestro objetivo es obtener un array `s` con los 10 valores del volumen de agua del deposito para cada dia `t`. El modelo va consistir simplemente en obtener `s[t]` como el volumen agua disponible (del dia anterior) y que entra al dia (`rain[t]`) menos el agua sale al dia por el desagüe (`q`).

![](util/modelo_deposito_2.png)

Vamos a ver el proceso de calculo paso a paso para comprenderlo un poco mejor.

1) Para `t = 0`
- La ecuacion para calcular el volumen de agua en el deposito seria:
```python
s[t] = s_ini + rain[t] - q
```
donde `s_ini` es el volumen de agua disponible inicialmente y que suponemos igual a 0.

Como han entrado 3 litros de agua de lluvia en un dia y el desagüe tiene capacidad de 3 litros al dia, entonces `q = 3`. Es decir, lo que entra `[rain[0]` es igual a lo que sale `q` y el deposito no almacena nada de agua, `s[0] = 0`.

Asi se escribiría en Python:

In [ ]:
t = 0 # definimos el dia
s_ini = 0 # definimos el volumen inicial de agua almacenada en el deposito
q = 3
s[t] = s_ini + rain[t] - q 

Al ejecutar la celda de arriba nos da un error porque el array `s` no está definido. Voy a intentarlo de nuevo pero ahora voy adefinir `s` con un valor inicial, por ejemplo 0.

In [ ]:
t = 0 # definimos el dia
s_ini = 0 # definimos el volumen inicial de agua almacenada en el deposito
q = 3
s = 0
s[t] = s_ini + rain[t] - q 

😓 De nuevo me da error...

⚡ ¿por qué?

Como hemos dicho arriba, `s` debe ser un array (es decir una estructura de valores) que contenga los valores del volumen del deposito para los 10 dias de la simulacion. Es decir `s[t]` **para t = 0, 1, ... 9**. Sin embargo, el valor inicial que le hemos dado a `s` ha sido un numero entero, en concreto 0 y por tanto Python no reconoce `s` como un array (o estructura de valores) sino como un numero entero. Y por tanto si le decimos que almacene un valor en el elemento `s[t]` Python no lo entiende ya que un numero entero es un valor unico, no una estructura de 10 valores distintos.  

Por tanto tenemos primero que crear un array inicial donde el modelo irá guardando el resultado para cada uno de los días `t` (o pasos de la simulación).

😕 Suena un poco raro ¿no?, necesitamos definir el resultado de la simulacion antes de hacer los calculos con el modelo??? Pues en realidad es algo muy común en programación y en particular cuando tenemos calculos con secuencias temporales en el que en cada paso temporal (time-step) los valores de las variables o arrays se van actualizando con nuevos valores. 

## Declaracion o inicializacion de arrays

A esto se le llama **declaracion o inicializacion de arrays** o tambien puedes encontrarlo como declaracion de variables. En esencia consiste en crear primero el recipiente vacio donde vamos a almacenar los resultados en cada paso de la simulacion (time-step).

Imagina que creamos una funcion para llenar la cubitera, por ejemplo llenando un hueco en cada paso (time-step). Pues la declaracion o inicializacion consistiria en crear primero una cubitera vacia y así indicarle a la función como es nuestra cubitera, es decir cuantos huecos hay (y que forma tienen). 

![](util/inicializacion_array.png)

Al crear nuestra cubitera vacía o array vacio, lo que le estamos indicando a Python es la cantidad de espacio que debe reservar en memoria para almacenar los resultados de nuestro modelo y ademas que tipos de datos vamos a almacenar. Para inicializar un array la mejor manera es calcular un array de ceros con la funcion de NumPy `zeros`.

In [ ]:
s = np.zeros([10,1]) # creamos un array de ceros de 10 filas y 1 columna

Si ahora intentamos de nuevo calcular y almacenar el resultado de la ecuacion en s[t]:

In [ ]:
t = 0 # definimos el dia
s_ini = 0 # definimos el volumen inicial de agua almacenada en el deposito
q = 3
s[t] = s_ini + rain[t] - q 

😊 ¡No da error! ¿ha cambiado ***s***?

In [ ]:
s

Vamos al siguiente dia de simulacion 

2) Para ***t = 1***

- Ahora nuestro volumen inicial va a ser el volumen del deposito el dia anterior: s[t-1]
- La prediccion de lluvia es rain[1] = 15 litros
- La ecuacion para calcular el volumen de agua en el deposito seria:
```python
s[t] = s[t-1] + rain[t] - q
```
![](util/modelo_deposito_4.png)

⚡ ¿Que valor tendria ***q***? ¿y ***s***?

In [ ]:
t = 1
q = 3
s[t] = s[t-1] + rain[t] - q 

¿podemos automatizar el calculo de `q`? Si el agua que entra al deposito es menor o igual a la capacidad del desague, entonces toda el agua que entra sale y no se alamacena nada en el deposito, asi q = rain[t]. En cambio, si llueve mas agua de la que se puede desalojar, entonces parte del agua queda almacenada y por el desague sale lo maximo posible `q_max`. En otras palabras, q seria el **mínimo** valor entre `rain[t]` y `q_max`

In [ ]:
q = np.min([rain[t], q_max])
q

De esta manera el calculo quedaria asi:

In [ ]:
t = 1
q = np.min([rain[t], q_max])
s[t] = s[t-1] + rain[t] - q 

¿Ha cambiado `s`?

In [ ]:
s

⚡ ¿ves algun problema? ¿como lo podemos solucionar?

In [ ]:
t = 1
q = np.min([rain[t], q_max])
s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 

In [ ]:
s

3) Para `t = 2`

- Nuestro volumen inicial va a ser el volumen del deposito el dia anterior: `s[t-1]`
- La prediccion de lluvia es `rain[2]` = 2 litros

![](util/modelo_deposito_5.png)

- El calculo del volumen de agua en el deposito seria:

In [ ]:
t = 2
q = np.min([rain[t], q_max])
s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 

¿Ha cambiado `s`?

In [ ]:
s

⚡ ¿ves algo raro?

Cuanta agua se ha desalojado?

In [ ]:
q

⚡ ¿por que se ha deslojado menos agua de la capacidad del desagüe `q_max`?

Para calcular `q` solo hemos tenido en cuenta el volumen agua entra ese dia en el deposito `rain[t]` pero ¿no deberiamos tener tambien en cuenta tambien el volumen de agua tenemos almacenada del dia anterior `s[t-1]`?

```python
q = np.min([s[t-1] +  rain[t], q_max])
```

De esta manera el calculo quedaria asi:

In [ ]:
t = 2
q = np.min([s[t-1] +  rain[t], q_max])
s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 

In [ ]:
q

In [ ]:
s

Utilizando las formulas que hemos obtenido arriba podemos repetir el proceso para cada `t` hasta t = 9. 

![](util/modelo_deposito_6.png)

Pero ¿y si mejor hacemos utilizamos una funcion bucle como `for` y nos ahorramos trabajo y tiempo?

In [ ]:
timesteps = np.arange(0,10,1) # creamos un array llamado timesteps con los valores que va tomar t en cada paso: 0,1,2 ...9 
s = np.zeros([10,1]) # Inicializamos el array s

for t in timesteps: # para t igual a cada uno de los valores en timesteps
    q = np.min([s[t-1] +  rain[t], q_max])
    s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 

De esta manera hacemos todos los calculos en una sola celda con unas pocas lineas de codigo. 

In [ ]:
s

Pues esto, simplemente **dos ecuaciones** que se ejecutan dentro de un bucle `for` es un **modelo** que simula el llenado y vaciado de un deposito de agua. Como veis un modelo puede ser muy simple y en realidad modelos mas complejos como los modelos hidrologicos no son mas que combinaciones de varios depositos.

![](util/ejemplos_modelos_hidrologicos.png)

Para no tener que estar copiando y pegando el codigo cada vez que lo queramos utilizar vamos a crear un funcion que contenga nuestro modelo. La funcion se llama ***modelo_deposito*** y tiene como inputs las caracteristicas del deposito `_max` y `q_max` y la prediccion de lluvias `rain`:

In [ ]:
def modelo_deposito(s_max, q_max, rain):
    timesteps = np.arange(0,10,1) # 
    s = np.zeros([10,1]) # Inicializamos el array s
    
    for t in timesteps:
        q = np.min([s[t-1] +  rain[t], q_max])
        s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 
    
    return s

Una vez creada nuestra funcion la podemos utilizar cuando queramos

In [ ]:
s = modelo_deposito(s_max, q_max, rain)

Podemos tambien cambiar los inputs de la funcion, si por ejemplo queremos probar un deposito diferente con volume maximo `s_max` mayor.

In [ ]:
s_max = 15
modelo_deposito(s_max, q_max, rain)

O si por ejemplo queremos utilizar una prediccion de la lluvia a mas largo plazo, por ejemplo para los proximos **15 dias** (en vez de 10)

In [ ]:
rain = np.array([3,15,2,2,6,0,1,0,0,5,3,9,8,2,0]) # prediccion de lluvia de 15 dias
modelo_deposito(s_max, q_max, rain)

⚡ ¿Veis algo raro en el resultado? ¿es el mismo resultado? ¿por que?

Si nos fijamos en el codigo del modelo vemos que tanto para definir `timesteps` como para inicializar `s` hemos usado 10 para que tuvieran los mismos elementos que la prediccion de la lluvia `rain` que era para 10 dias. 

```python
timesteps = np.arange(0,10,1)
s = np.zeros([10,1])
```
Sin embargo ahora, esto ha cambiado y deberiamos cambiar el 10 por 15. Pero y si despues queremos utilizar una prediccion para 5 dias o para 20, ¿tenemos que cambiar cada vez este valor manualmente? ¿y si contamos el numero de elementos de `rain`, lo guardamos como una variable `T` y la ponemos en el lugar de `10`?

In [ ]:
def modelo_deposito(s_max, q_max, rain):
    T = np.size(rain)
    
    timesteps = np.arange(0,T,1) # 
    s = np.zeros([T,1]) # Inicializamos el array s
    
    for t in timesteps:
        q = np.min([s[t-1] +  rain[t], q_max])
        s[t] = np.min([s[t-1] + rain[t] - q, s_max]) 
    
    return s

In [ ]:
rain = np.array([3,15,2,2,6,0,1,0,0,5,3,9,8,2,0]) # prediccion para 15 dias
modelo_deposito(s_max, q_max, rain)

In [ ]:
rain = np.array([3,15,2,2,6]) # prediccion para 5 dias
modelo_deposito(s_max, q_max, rain)

Creo que ya podemos decir que nuestro modelo funciona suficientemente bien. Como habeis podido ver para nuestro modelo hemos necesitado crear un funcion con `def` y aplicar un bucle `for` y varias funciones de Numpy: `size`, `arange`, `zeros`, `min`.

⚡ ¿se os ocurre otra manera de escribir el modelo? ¿se podria definir la ecuacion
```python
q = np.min([s[t-1] +  rain[t], q_max])
```
de otra manera, por ejemplo usando operaciones condicionales `if`, `else`?

In [ ]:
if s[t-1] +  rain[t] > q_max:
    
else:
    

⚡ Ahora cread un nuevo modelo que se llame `modelo_deposito_2` que sea igual que `modelo_deposito` pero con la nueva forma de calcular `q` que hemos definido arriba